## Set up

In [3]:
! pip install -U langchain_community langchain-pinecone tiktoken chromadb langchain langchainhub sentence_transformers transformers
! pip install pinecone-client
! pip install llama-index>=0.9.31 pinecone-client>=3.0.0
! pip install --upgrade tensorflow
! pip install -U langchain-huggingface
! pip install pypdf
! pip install langserve
! pip install llamaapi -q
! pip install tf-keras
! pip install groq
! pip install langchain-groq

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable


## Data Ingestion

In [4]:
# import os
# from langchain_community.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from sentence_transformers import SentenceTransformer
# from langchain.vectorstores import Pinecone

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
# PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")
# PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

# ## Ingest code - 
# loader = PyPDFLoader("元宇宙.pdf")
# data = loader.load()

# # Split
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# all_splits = text_splitter.split_documents(data)


In [5]:
# print(all_splits[0])

In [6]:
# from sentence_transformers import SentenceTransformer
# from langchain_pinecone import PineconeVectorStore

# # Initialize the SentenceTransformer model
# embedding_model = SentenceTransformer("intfloat/multilingual-e5-large")

# # Add to vectorDB
# vectorstore = PineconeVectorStore.from_documents(
#     documents=all_splits, 
#     embedding=embedding_model,
#     index_name=PINECONE_INDEX_NAME
# )
# retriever = vectorstore.as_retriever()

## Chain Development

In [7]:
import os
import json
from operator import itemgetter
from typing import List, Tuple
from langchain_pinecone import PineconeVectorStore
from langchain_groq import ChatGroq
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    format_document,
)
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

if os.environ.get("PINECONE_API_KEY", None) is None:
    raise Exception("Missing `PINECONE_API_KEY` environment variable.")

if os.environ.get("PINECONE_ENVIRONMENT", None) is None:
    raise Exception("Missing `PINECONE_ENVIRONMENT` environment variable.")


2024-07-03 13:59:05.681324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-03 13:59:05.936330: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-03 13:59:05.938041: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-03 13:59:06.352830: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-03 13:59:08.310705: W tensorflow/compiler/tf

In [8]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
llama = ChatGroq(model="llama3-8b-8192",temperature=0)
vectorstore = PineconeVectorStore.from_existing_index(
    PINECONE_INDEX_NAME, embedding
)
retriever = vectorstore.as_retriever()


In [9]:
# Prompt Templates for Answer Synthesis
# Ask: Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)
print(CONDENSE_QUESTION_PROMPT)


input_variables=['chat_history', 'question'] template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:'


In [13]:
# Answer: RAG answer synthesis prompt-context and format, chat history and user question
template = """你是一位在中国元宇宙领域有深厚专业知识的教授。你的目标是与元宇宙爱好者、学者以及业界和学术界的专业人士分享你的知识。你应该耐心且正式地讲话，总是使用案例研究来解释你的观点。仅根据以下上下文用中文回答问题:
<context>
{context}
</context>"""
ANSWER_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{question}"),
    ]
)

# Chain for Conversational Retrieval
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")
print(ANSWER_PROMPT)
print(DEFAULT_DOCUMENT_PROMPT)

input_variables=['chat_history', 'context', 'question'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='你是一位在中国元宇宙领域有深厚专业知识的教授。你的目标是与元宇宙爱好者、学者以及业界和学术界的专业人士分享你的知识。你应该耐心且正式地讲话，总是使用案例研究来解释你的观点。仅根据以下上下文用中文回答问题:\n<context>\n{context}\n</context>')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]
input_variables=['page_content'] template='{page_content}'


In [14]:
# Combine the documents by using document_separator
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# Human Message+ AI Message[HumanMessage(content=''), AIMessage(content=""), ...]
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

# Define chat history and user input
class ChatHistory(BaseModel):
    chat_history: List[Tuple[str, str]] = Field(..., extra={"widget": {"type": "chat"}})
    question: str
    
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | llama
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(itemgetter("question")),
)

_inputs = RunnableParallel(
    {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "context": _search_query | retriever | _combine_documents,
    }
).with_types(input_type=ChatHistory)

chain = _inputs | ANSWER_PROMPT | llama | StrOutputParser()



In [15]:
chain.invoke({'question':'元宇宙有几个发展阶段','chat_history':[]})

'根据同济大学臧志彭教授的元宇宙理论研究成果，元宇宙可以划分为三个发展阶段：\n\n1. 元宇宙1.0：虚实基础层全面协同建构的生产力变革期。这一阶段是元宇宙首先需要建构的是能够与现实世界互联融通的虚拟世界。如果虚拟世界还未建设出来，自然也就无法去谈所谓的虚实融合。这一阶段的核心目标是在元宇宙构建出符合人类梦想与欲望要求的三类元宇宙新世界：第一类是复刻现实世界的虚拟世界；第二类是通过创意构想创造的创新虚拟世界；第三类是与现实世界深度联动交互的虚实融合世界。\n2. 元宇宙2.0：新型生产关系与经济规律成型的元宇宙阶段。这一阶段是人类新一代生存方式导入阶段；同时也是人类新一代生存方式发展阶段。在这一阶段，元宇宙将经历数字孪生、虚拟原生、虚实融生和虚实联动四大阶段。\n3. 元宇宙3.0：全球新治理秩序与政治格局形成的元宇宙阶段。这一阶段是人类新一代生存方式成熟阶段；同时也是世界格局重塑阶段。在这一阶段，元宇宙将出现各个主体之间的新型矛盾冲突与力量博弈，进而形成新的全球治理秩序和世界格局。\n\n需要注意的是，这三个阶段并不是严格的阶段划分，而是人类元宇宙新数字文明未来演进的三个主要阶段。'